In [141]:
import FinanceDataReader as fdr
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import pickle
import time

In [142]:
dt_list1 = ['2023-01-02','2023-01-03','2023-01-04','2023-01-05','2023-01-06','2023-01-09','2023-01-10','2023-01-11','2023-01-12','2023-01-13','2023-01-16',\
            '2023-01-17','2023-01-18','2023-01-19','2023-01-20','2023-01-25','2023-01-26','2023-01-27','2023-01-30','2023-01-31']
dt_list2 = ['2023-02-01','2023-02-02','2023-02-03','2023-02-06','2023-02-07','2023-02-08']
dt_list = dt_list1 + dt_list2 

In [143]:
df_all = pd.DataFrame()

for dt in dt_list:
    kq_yymmdd = pd.read_pickle('kq_selection_' + str(dt) + '.pkl')['name']
    for code in list(kq_yymmdd.index):
        start_dt = dt
        end_dt = datetime.datetime.strptime(dt, '%Y-%m-%d') + BDay(10)
        df = fdr.DataReader(code, start=start_dt, end=end_dt) 
        df = df.head(6)
        df['code'] = code
        df['name'] = kq_yymmdd.loc[code]
        df['select_dt'] = dt 
        df = df.reset_index().set_index('select_dt')[['Date','code','name','code','Open','High','Low','Close','Volume']]
        df_all = pd.concat([df_all, df], axis=0)

In [82]:
df_all.to_excel('backtesting.xlsx')

In [144]:
c1 = df_all.groupby(['select_dt','name'])[['name','Close']].first(1)
c2 = df_all.groupby(['select_dt','name'])[['Open']].nth(1)
c3 = df_all.groupby(['select_dt','name'])[['High']].nth([1,2,3,4,5]).groupby(['select_dt','name']).max()
c4 = df_all.groupby(['select_dt','name'])[['Low']].nth([1,2,3,4,5]).groupby(['select_dt','name']).min()
c_all = pd.concat([c1, c2, c3, c4], axis=1)
c_all['max_return'] = c_all['High']/c_all['Open']
c_all['min_return'] = c_all['Low']/c_all['Open']

In [131]:
c_all.to_excel('hyungbong.xlsx')

In [139]:
# b1 =  df_all.groupby(['select_dt','name'])[['name','Low','High']].first(1).apply(lambda x: x['High'] - x['Low'], axis=1)
# b2 = df_all.groupby(['select_dt','name'])[['Open','High']].nth(1)
# b3 = df_all.groupby(['select_dt','name'])['Open'].nth(2)

In [170]:
price_range = df_all.groupby(['select_dt','name']).first()[['Low','High']].apply(lambda x: (x['High'] -x['Low']), axis=1)

In [173]:
open_price = df_all.groupby(['select_dt','name']).nth(1)['Open'] 
min_price

Close   Open   High    Low  max_return  min_return
select_dt  name                                                       
2023-01-02 CBI        426    431    436    376    1.011601    0.872390
           코스나인       630    627    638    590    1.017544    0.940989
           크린앤사이언스   9770   9560  11950   9560    1.250000    1.000000
2023-01-03 MDS테크     1883   1825   2990   1735    1.638356    0.950685
           나무가      15700  15550  16400  14600    1.054662    0.938907
...                   ...    ...    ...    ...         ...         ...
2023-02-06 엠플러스     10360  10420  12140  10400    1.165067    0.998081
2023-02-07 디앤씨미디어   24350  24350  24950  22300    1.024641    0.915811
           오리콤      14170  14250  14840  13000    1.041404    0.912281
2023-02-08 코맥스       3815   3850   5370   3800    1.394805    0.987013
           티라유텍     10190  10110  10240   9730    1.012859    0.962413

[118 rows x 6 columns]

In [177]:
c_all[c_all['min_return']>0]['min_return'].describe()

count    117.000000
mean       0.959452
std        0.032561
min        0.847744
25%        0.945299
50%        0.965997
75%        0.983425
max        1.000000
Name: min_return, dtype: float64

In [178]:
c_all[c_all['max_return']>0]['max_return'].describe()

count    118.000000
mean       1.103320
std        0.128316
min        1.000000
25%        1.024501
50%        1.051128
75%        1.134226
max        1.664753
Name: max_return, dtype: float64